In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import os
from load_data import load_data
import cv2 
import data_utils
import os

#####Transformation Matrix
num_points = 429

M = np.zeros((4,4,num_points))
a = np.array([0,0,0,1])
vel = np.zeros((3,num_points))
ang = np.zeros((3,num_points))
#point_cloud = np.zeros((3,num_points))
sample_start = np.zeros(num_points)
sample_camera = np.zeros(num_points)
sample_end = np.zeros(num_points)

time_start = 'data/problem_4/velodyne_points/timestamps_start.txt'
time_camera = 'data/problem_4/velodyne_points/timestamps.txt'
time_end = 'data/problem_4/velodyne_points/timestamps_end.txt'

angle_start_velo = np.zeros(num_points)
for i in range(num_points):
    file_index = str(i)
    str_0 = (10-len(file_index))*"0"
    data_path = os.path.join('data/problem_4/oxts/data/', str_0 + file_index+ '.txt')
    vel[:,i] = data_utils.load_oxts_velocity(data_path)
    ang[:,i] = data_utils.load_oxts_angular_rate(data_path)
    sample_start[i] = data_utils.compute_timestamps(time_start, i)
    sample_camera[i] = data_utils.compute_timestamps(time_camera, i)
    sample_end[i] = data_utils.compute_timestamps(time_end, i)
    #3. Compute translation using angular velocity and velocity
    pos_xy_cam = -vel[0:2, i]*(sample_camera[i]-sample_start[i])
    #4.Compute rotation using pos and angular velocity
    teta = -ang[2, i]*(sample_camera[i]-sample_start[i])
    R = np.array([[np.cos(teta), -np.sin(teta), 0],
                [np.sin(teta), np.cos(teta), 0],
                [0,             0,     1]])
    T = np.array([[pos_xy_cam[0]],
                [pos_xy_cam[1]],
                [0]])

    inter = np.hstack((R,T))
    inter = np.vstack((inter, a))
    M[:,:,i] = inter
    
    w_velodyne = 2*np.pi/(sample_end[i]-sample_start[i])
    ang_no_corr = w_velodyne*(sample_camera[i]-sample_start[i])
    angle_start_velo[i] = ang_no_corr + teta
         
    ####Velodyne points
    file_index = str(i)
    str_0 = (10-len(file_index))*"0"
    data_velodyne = os.path.join('data/problem_4/velodyne_points/data/', str_0 + file_index+ '.bin')
    point_cloud = data_utils.load_from_bin(data_velodyne)
    alpha = np.zeros(point_cloud.shape[0])
    for j in range(point_cloud.shape[0]):
        hyp = np.sqrt(point_cloud[j,0]**2+point_cloud[j,1]**2)
        alpha[j] = np.arcsin(point_cloud[j,1]/hyp)
        cos = point_cloud[j,0]/hyp
        if cos < 0:
            alpha[j] = alpha[j] + np.pi
        if alpha[j] < 0:
            alpha[j] = 2*np.pi + alpha[j]
        alpha[j] = alpha[j] - angle_start_velo[i]
    np.argmin(dist)
    #alpha = np.sort(alpha)
    if i ==1:
        print(alpha)
        

[-3.13886121 -3.13875788 -3.13873458 ...  3.14425787  3.14429495
  3.14429889]
